<a href="https://colab.research.google.com/github/LorenzoCiappetta/Features-mapping-using-Two-Frame-Structure-from-motion-and-AI-detection/blob/main/Detection_and_Sfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2 as cv
import glob
from matplotlib import pyplot as plt
import math

#for Ransac
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform, AffineTransform

In [ ]:
#classes used to make the map

#feature represents the streetlamps, with their positions and the photos in which they appear
class Feature:
  def __init__(self, src_image_name, image_corners_coords ,space_coords):
    self.images = []#set of the names of image files where this feature is present and where is present in the image
    self.images.append((src_image_name, image_corners_coords))
    self.space_coords = space_coords #estimate of spatial coordinates of feature (possibly in meters)

  def __str__(self):
    return f"feature found in {self.images}, likely to be at {self.space_coords}"

  #same feature observed in another image
  def add_image(self, new_image_name, image_corners_coords):
    if (new_image_name, image_corners_coords) not in self.images:
      self.images.append((new_image_name, image_corners_coords))

#status represents the images, with the position from which they're taken and the features which appear in them
class Status:

  def __init__(self, src_image_name, kps, des ,camera_coords, camera_orientation):
    self.features = []#from each position there are a number of observable features (always street lamps)
    self.image = src_image_name
    self.kps = kps
    self.des = des
    self.camera_coords = camera_coords
    #on x,y graph first status always has (0,1) orientation
    self.camera_orientation = camera_orientation


  def add_feature(self, feature, feature_corners):
    self.features.append((feature, feature_corners))

#Map represent the map of statuses and feaures, they are contained in lists
class Map:

  def __init__(self):
    self.positions = []
    self.features = []


  def add_feature(self, feature):
    self.features.append(feature)


  def add_status(self, status):
    self.positions.append(status)

  def print(self):
    print('number of features in map = ', len(self.features))
    print('number of statuses in map = ', len(self.positions))
    maxx = 0
    maxy = 0
    minx = 9999
    miny = 9999
    xf = []
    yf = []
    xs = []
    ys = []
    for f in self.features:
      a,b = f.space_coords

      if a > maxx:
        maxx = a
      if b > maxy:
        maxy = b
      if a < minx:
        minx = a
      if b < miny:
        miny = b

      xf.append(a)
      yf.append(b)

    c,d = 0,0
    for s in self.positions:
      print('number of features visible from image ', s.image, ' = ', len(s.features))
      a,b = s.camera_coords

      print(np.sqrt((a-c)**2+(b-d)**2))

      c = a
      d = b

      if a > maxx:
        maxx = a
      if b > maxy:
        maxy = b
      if a < minx:
        minx = a
      if b < miny:
        miny = b

      xs.append(a)
      ys.append(b)

    max = 0
    min = 0
    if maxx > maxy:
      max = maxx
    else:
      max = maxy

    if minx > miny:
      min = minx
    else:
      min = miny

    plt.plot(xs, ys, 'bo')
    plt.plot(xf,yf,'ro')
    plt.axis('equal')
    plt.show()

In [ ]:
#Miscellaneous and auxiliaries

#Used to remove ambiguous matches in 'confront scenes'
def remove_same_point_correspondences(pts1, pts2):
  dt = np.dtype(float,float)

  f,g = np.unique(np.array(pts1, dtype=dt),return_counts= True, axis = 0)
  h,l = np.unique(np.array(pts2, dtype=dt),return_counts= True, axis = 0)

  f = [tuple(f[p]) for p in range(len(f)) if g[p] > 1]

  h = [tuple(h[p]) for p in range(len(h)) if l[p] > 1]

  pts1 = [pts1[p] for p in range(len(pts1)) if pts2[p] not in h]
  pts2 = [pts2[p] for p in range(len(pts2)) if pts2[p] not in h]

  pts2 = [pts2[p] for p in range(len(pts2)) if pts1[p] not in f]
  pts1 = [pts1[p] for p in range(len(pts1)) if pts1[p] not in f]

  return pts1, pts2

In [ ]:
#Operations on single image

#returns the coords of the corners of the box created by the detector
def retrieve_box_corners(img, description_path):
  h,w = 0, 0
  if len(img.shape) == 2:
    h, w = img.shape
  else:
    h, w, _ = img.shape

  res = []
  file = open(description_path, "r")
  line = file.readline()

  while line != "":
    data = line.split()
    x_center = float(data[1])
    y_center = float(data[2])
    width = float(data[3])
    height = float(data[4])

    x_center *= w
    y_center *= h
    width *= w
    height *= h

    corner1 = (round(x_center - width/2), (round(y_center - height/2)))
    corner2 = (round(x_center + width/2), (round(y_center - height/2)))
    corner3 = (round(x_center - width/2), (round(y_center + height/2)))
    corner4 = (round(x_center + width/2), (round(y_center + height/2)))

    figure = [corner1, corner2, corner3, corner4]
    res.append(figure)

    line = file.readline()
  file.close()
  return res


#returns the sizes of the box created by the detector
def retrieve_box_sizes(img, description_path):
  h,w = 0, 0
  if len(img.shape) == 2:
    h, w = img.shape
  else:
    h, w, _ = img.shape

  res = []
  file = open(description_path, "r")
  line = file.readline()

  while line != "":
    data = line.split()
    x_center = float(data[1])
    y_center = float(data[2])
    width = float(data[3])
    height = float(data[4])

    x_center *= w
    y_center *= h
    width *= w
    height *= h

    figure = [x_center, y_center, width, height]
    res.append(figure)

    line = file.readline()
  file.close()
  return res


#returns only keypoints fuond inside the box found by the detector
def is_point_in_border(point, corners, x_extension = 0, y_extension = 0):
  min_x = corners[0][0] - x_extension
  max_x = corners[1][0] + x_extension
  min_y = corners[0][1] - y_extension
  max_y = corners[2][1] + y_extension

  x = point.pt[0]
  y = point.pt[1]

  if x >= min_x and x <= max_x and y >= min_y and y <= max_y:
    return True

  return False

In [ ]:
#Main functions for feature matching and Sfm

#calibrates camrea, returns intrinsic matrix
def calibrate_camera(chessboard_path, suffix='.jpg'):#from opencv tutorial website
  chessboard_path += '/*' + suffix

  # termination criteria
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

  # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
  objp = np.zeros((6*7,3), np.float32)
  objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

  # Arrays to store object points and image points from all the images.
  objpoints = [] # 3d point in real world space
  imgpoints = [] # 2d points in image plane.

  images = glob.glob(chessboard_path)

  for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
      objpoints.append(objp)

      corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
      imgpoints.append(corners2)

      #cv.drawChessboardCorners(img, (7,6), corners2, ret)
      #plt.imshow(img)

  ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
  return mtx, dist, rvecs, tvecs


#uses SIFT to detect keypoints
def detect_keypoints(img):
  # Initiate SIFT detector
  sift = cv.SIFT_create()

  # find the keypoints and descriptors with SIFT
  kps, des = sift.detectAndCompute(img,None)

  return kps, des


#removes keypoints outside a box
def filter_keypoints(kps, des, corners):

  ret_kps = []
  ret_des = []
  for i in range(len(kps)):
    if is_point_in_border(kps[i], corners, 200, 200):
      ret_kps.append(kps[i])
      ret_des.append(des[i])

  return tuple(ret_kps), np.array(ret_des)


#confronts features in images, two images with many matchings have high chance of portaiyng same streetlamp
def confront_scenes(kp1, des1, kp2, des2):

  #FLANN matches
  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks = 50)

  flann = cv.FlannBasedMatcher(index_params, search_params)

  matches = flann.knnMatch(des1,des2,k=2)

   # Apply ratio test
  pts1, pts2 = [],[]
  for m,n in matches:
    if m.distance < 0.75*n.distance:
      pts1.append(kp1[m.queryIdx].pt)
      pts2.append(kp2[m.trainIdx].pt)

  #remove dupliacates
  pts1,pts2 = remove_same_point_correspondences(pts1, pts2)

  pts1,pts2 = np.float32(pts1), np.float32(pts2)

  if len(pts1) < 5:#cannot apply RANSAC
    return [],[]

  #apply Ransac
  model, inliers = ransac(
      (pts1,pts2),
      AffineTransform, min_samples=4,
      residual_threshold=8, max_trials=10000
  )

  n_inliers = np.sum(inliers)
  print(n_inliers)
  if n_inliers is None or n_inliers == 0:
    return [], []

  inlier_pts1 = [cv.KeyPoint(point[0], point[1], 1) for point in pts1[inliers]]
  inlier_pts2 = [cv.KeyPoint(point[0], point[1], 1) for point in pts2[inliers]]

  placeholder_matches = [cv.DMatch(idx, idx, 1) for idx in range(n_inliers)]

  pts1 = np.float32([ inlier_pts1[m.queryIdx].pt for m in placeholder_matches ]).reshape(-1, 2)
  pts2 = np.float32([ inlier_pts2[m.trainIdx].pt for m in placeholder_matches ]).reshape(-1, 2)

  return pts1, pts2

#retrieves the Essential matrix E and uses it to fintìd rotation matrix R and translation vector t
def get_rotation_and_translation(intrinsic_matrix, pts1, pts2):

  pts1 = np.int32(pts1)
  pts2 = np.int32(pts2)
  E, mask = cv.findEssentialMat(pts1, pts2, intrinsic_matrix,cv.RANSAC)

  #recover R and t and
  #check for cheirality
  retval, R, t, mask = cv.recoverPose(E, pts1, pts2, intrinsic_matrix, mask)
  return R,t

In [ ]:
#Just Geometry

#given R, t and distances, returns where the camera is in relation to previous position at (0,0) looking towards increasing y
def get_camera_position(R, t, distance_of_cameras,direction_of_first_camera = np.array([0,0,1])):
  #where it is
  c = -R.T @ t
  c = np.array([c[0],c[2]])
  c = (c/np.linalg.norm(c))*distance_of_cameras
  c = np.reshape(c,(2,))
  #where it's looking
  obj = R.T @ direction_of_first_camera
  obj = np.array([obj[0],obj[2]])
  #normalize vector because it's just for direction
  obj = (obj/np.linalg.norm(obj))

  return c, obj

#takes coordinates with camera at (0,0) turns them into map coordinates
#all inputs are vectors which represent a direction
def get_coordinates_in_map(translation, camera_orientation, real_coordinates, real_orientation):

  translation, camera_orientation, real_coordinates, real_orientation = np.array(translation), np.array(camera_orientation), np.array(real_coordinates), np.array(real_orientation)

  x = real_orientation[0]
  y = real_orientation[1]

  rot = None

  if y != 0:
    real_angle = np.arctan(x/y)
    rot = np.array([[np.cos(real_angle),-np.sin(real_angle)],[np.sin(real_angle),np.cos(real_angle)]])
  else:
    if x > 0:
      rot = np.array([[0,-1],[1,0]])
    else:
      rot = np.array([[0,-1],[1,0]])

  real_translation = translation @ rot

  real_translation[0] += real_coordinates[0]
  real_translation[1] += real_coordinates[1]

  real_camera_orientation =  camera_orientation @ rot

  return real_translation, real_camera_orientation


#given two points returns a third on same line at a distance from the second point
def next_point_in_line(pt1, pt2, distance):
  x = pt1[0]-pt2[0]
  y = pt1[1]-pt2[1]

  signx = 1
  signy = 1

  if x < 0:
    signx = -1
  if y < 0:
    signy = -1

  if x == 0.0:
    if y < 0:
      x = 0
      y = -distance
    else:
      x = 0
      y = distance
  else:
    angle = np.arctan(y/x)
    x = np.cos(angle)*distance
    y = np.sin(angle)*distance

  if signx*x < 0:
    x *= -1
  if signy*y < 0:
    y *= -1

  pt3 = (x+pt1[0],y+pt1[1])
  return pt3

#returns coordinates of feature for a camera at (0,0) facing direction_of_camera
def get_feature_coords(distance_from_camera, offset_from_camera, direction_of_camera):

  np.array(direction_of_camera)
  rot = np.array([[0,-1],[1,0]])
  direction_of_offset = direction_of_camera @ rot

  distance = direction_of_camera/np.linalg.norm(direction_of_camera) * distance_from_camera
  offset = direction_of_offset/np.linalg.norm(direction_of_offset) * offset_from_camera

  return distance + offset


#to find position of object
def triangle_similarity(intrinsic_matrix, pixel_height, real_height, center):
  fx = intrinsic_matrix[0][0]
  fy = intrinsic_matrix[1][1]
  cx = intrinsic_matrix[0][2]
  cy = intrinsic_matrix[1][2]
  x = center[0]
  y = center[1]
  distance = (real_height * fy) / pixel_height
  offset = ((x-cx)*distance) / fx
  return distance, offset

In [ ]:
#aux functions to create_map

#used to check if a feature is present in a previous status
def confront_feature_and_status(intrinsic_matrix, prev_status, new_status, corners2, feature_position, position, discarted_features):
  matches_in_same_image={}

  imgname1 = prev_status.image
  imgname2 = new_status.image

  for other_f in prev_status.features:

    corners1 = other_f[1]
    f = other_f[0]

    if f not in discarted_features:

      kp1,des1 = filter_keypoints(prev_status.kps, prev_status.des, corners1)
      kp2, des2 = filter_keypoints(new_status.kps, new_status.des, corners2)
      pts1,pts2 = confront_scenes(kp1, des1, kp2, des2 )

      if len(pts1) <= 5:
        print('no match between:')
        print(imgname1,':',corners1)
        print(imgname2,':',corners2)
        discarted_features.append(f)

      else:

        matches_in_same_image[f] = corners1

    else:
      print('saving some time')

  best_feature = None
  best_distance = 9999
  c = None

  #filters all found matches leaving only the closest one
  for f_match in matches_in_same_image:
    pf_old = f_match.space_coords

    distance = np.linalg.norm(np.array(pf_old) - np.array(feature_position))
    if distance < best_distance:
      best_feature = f_match
      best_distance = distance
      c = matches_in_same_image[f_match]

  if best_feature is not None:
    print('found match between:')
    print(imgname1,':', c)
    print(imgname2,':',corners2)

  return best_feature

#given the positions found for the same streetlamp finds a better estimate
def get_better_position_estimate(plausible_positions):
  #for now it is just the average
  res = [0,0]
  n = len(plausible_positions)
  for xy in plausible_positions:
    res[0]+=xy[0]
    res[1]+=xy[1]

  res[0]/=n
  res[1]/=n
  return tuple(plausible_positions.pop())#tuple(res)

def get_better_orientation_estimate(plausible_orientations):
  #for now it is just the average
  res = [0,0]
  n = len(plausible_orientations)
  for xy in plausible_orientations:
    res[0]+=xy[0]
    res[1]+=xy[1]

  res[0]/=n
  res[1]/=n
  return tuple(res)

In [ ]:
#this function creates the map
def create_map3(labels_path, imgs_path, distance, street_lamp_height, intrinsic_matrix, steps = 99, suffix = '.jpg'):

  steps_count = 0
  map = Map()
  curr_position = (0,0)
  prev_position = (0,0)
  prev_orientation = (0,1)
  prev_status = None
  d = 0

  #reading the files
  length1 = len(imgs_path)
  length2 = len(labels_path)
  images = glob.glob(imgs_path + "/*" + suffix)
  images.sort()

  labels = glob.glob(labels_path + "/*.txt")
  labels.sort()

  #it also work if the image set is in the reversed order
  #images.reverse()
  #labels.reverse()

  #scans one image at a a time
  for imgname in images:

    #check condition to stop early
    print(steps_count)
    if steps_count >= steps:
      return map
    steps_count +=1

    #for each image searches its labels
    name = imgname[length1 + 1: (len(imgname)- len(suffix))]
    if len(labels) > 0:
      text = labels[0]
      text = text[length2 + 1: (len(text)- len('.txt'))]
    else:
      text = ''

    lbls = ''

    if name == text :
      lbls = labels.pop(0)

    #reads image and finds keypoints
    curr_img = cv.imread(imgname, cv.IMREAD_GRAYSCALE)
    kps, des = detect_keypoints(curr_img)
    #print(len(kps))

    if prev_status is not None:#if not first image in sequence

      #extracts distance from previous image
      if isinstance(distance, list):
        d = distance.pop(0)
      else:
        d = distance

      #confronts images to find matches
      pts1, pts2 = confront_scenes(prev_status.kps, prev_status.des, kps, des)

      #print('found:', len(pts1),'points')

      if len(pts1) <= 5:
        print('not enough points found')
        #not enough points, it has to guess position...
        new_position = next_point_in_line(curr_position, prev_position, d)
        new_orientation = prev_orientation

        prev_position = curr_position
        curr_position = new_position
        prev_orientation = new_orientation

      else:
        #finds where the camera is
        R,t = get_rotation_and_translation(intrinsic_matrix, pts1,pts2)
        c,ori = get_camera_position(R,t,d)

        #this is the position of the camera
        new_position, new_orientation = get_coordinates_in_map(c, ori, curr_position, prev_orientation)

        prev_position = curr_position
        curr_position = new_position
        prev_orientation = new_orientation

      #creates new status with the found position
      new_status = Status(imgname, kps, des, curr_position, prev_orientation)

      if lbls != '':#there are streetlamps in image
        curr_features_sizes = retrieve_box_sizes(curr_img, lbls)
        curr_features_corners = retrieve_box_corners(curr_img, lbls)

        #in case two features in the same image match the same one in another image
        ambiguous_matches = {}
        #list of new features
        left_out_features = []

        #for every lamp
        for f in curr_features_sizes:

          #calculates where lamp is
          height = f[3]
          center = (f[0], f[1])
          corners = curr_features_corners.pop(0)

          distance_from_camera, offset_from_camera = triangle_similarity(intrinsic_matrix, height, street_lamp_height, center)
          print('object is far away:', distance_from_camera)
          if distance_from_camera > 30:
            print('ignoring objects too far away')
            continue

          #this is the position of the feature
          pf_new = get_feature_coords(distance_from_camera, offset_from_camera, prev_orientation)
          pf_new[0] += curr_position[0]
          pf_new[1] += curr_position[1]

          #scan all previous images to find if the feature is alredy present
          found_match = False
          discarted_features = []
          for s in reversed(map.positions):

            f_matched = confront_feature_and_status(intrinsic_matrix, s, new_status, corners, pf_new, curr_position, discarted_features)
            if f_matched is not None:
              old_coords = f_matched.space_coords

              error = np.linalg.norm(np.array(old_coords) - np.array(pf_new))
              print('error:', error)
              #if the lamp is too far away from where it should be likely the match is wrong
              if error < 10:
                #if this is the only one that matches
                if f_matched not in ambiguous_matches:

                  ambiguous_matches[f_matched] = (pf_new, corners)
                  found_match = True
                  break

                else:
                  pf_other = ambiguous_matches[f_matched][0]
                  corners_other = ambiguous_matches[f_matched][1]
                  other_error = np.linalg.norm(np.array(old_coords) - np.array(pf_other))

                  #if one had already matched only keeps the closest
                  if error < other_error:
                    print('corrected ambiguity')
                    ambiguous_matches[f_matched] = (pf_new, corners)
                    left_out_features.append((pf_other, corners_other))
                    found_match = True
                    break

                  else:
                    print('match discarted for ambiguity')

              else:
                print('distance of matched features is above threshold, match is likely wrong')

          if not found_match:
            #new feature...
            print('found new feature')
            left_out_features.append((pf_new, corners))

        #finds a more accurate position for old features
        for m in ambiguous_matches:
          old_coords = m.space_coords
          pf_new = ambiguous_matches[m][0]
          corners_new = ambiguous_matches[m][1]
          new_coords = get_better_position_estimate([old_coords,pf_new])
          m.space_coords = new_coords
          m.add_image(imgname, corners_new)
          new_status.add_feature(m, corners_new)

        #inserts new features in map
        for f in left_out_features:
          f_new = Feature(imgname, f[1], f[0])
          new_status.add_feature(f_new, f[1])
          map.add_feature(f_new)

      #adds image in map
      map.add_status(new_status)
      prev_status = new_status

    else:#this is first image
      #starts at (0,0)
      new_status = Status(imgname, kps, des, curr_position, prev_orientation)

      if lbls != '':#there are features

        curr_features_sizes = retrieve_box_sizes(curr_img, lbls)
        curr_features_corners = retrieve_box_corners(curr_img, lbls)

        for f in curr_features_sizes:
          height = f[3]
          center = (f[0], f[1])
          corners = curr_features_corners.pop(0)
          #finds where features are
          distance_from_camera, offset_from_camera = triangle_similarity(intrinsic_matrix, height, 7.5, center)
          print('object is far away:', distance_from_camera)
          if distance_from_camera > 50:
            print('ignoring objects too far away')
            continue

          #all features are in this case new feature...
          print('found new feature')
          pf_new = get_feature_coords(distance_from_camera, offset_from_camera, prev_orientation)
          pf_new[0] += curr_position[0]
          pf_new[1] += curr_position[1]
          f_new = Feature(imgname, corners, pf_new)
          new_status.add_feature(f_new, corners)
          map.add_feature(f_new)

      map.add_status(new_status)
      prev_status = new_status

  return map

In [ ]:
#to detect lamps must use the detect.py program from terminal
#it is situated inside the directory yolov9
#make sure the --save-txt instruction is present, it will save the labels in the yolov9 directory, they're needed in the next cell
%cd /content/'path to yolov9'/yolov9
!python detect.py --weights /content/'path to yolov9'/yolov9/runs/train/exp3/weights/best.pt --conf 0.40 --source /content/'path to the streetlamp pictures' --device cpu --save-txt

In [ ]:
#if the intrinsic matrix is not known it is necessary to calibrate first, insert the path to the photos of checkerboard pattern and their file extension (png, jpg, JPG etc.)
#sometimes calibration can require a large amount of photos, if it does not work at first probably it needs more pictures
mtx, _, _, _ = calibrate_camera('here goes the path of the directory containing chessboard photos', suffix = 'here goes the file extension')
print(mtx)

#in order to create the map it needs the path to the labels and the images, the distance between each picture (suggested 1m), the height of the streetlamps (may or may not be 7), and the matrix
map = create_map3('here goes the path to the labels generated by the detection', 'here goes the path to the images taken (the ones without the bounding box)', 1, 7, mtx, suffix = '.jpg')
#shows the map
map.print()